# Word tagging

Annotate relevant tags in texts. Example: name entities.



In [1]:
from __future__ import print_function

import os 
import sys

import numpy as np
import tensorflow as tf 

print(tf.__version__)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


1.0.0


## Dataset transformation

In [2]:
# Read data
import pickle
import sys

atis_file = '/home/ubuntu/data/training/text/atis/atis.pkl'

with open(atis_file,'rb') as f:
    if sys.version_info.major==2:
        train, test, dicts = pickle.load(f) #python2.7
    else:
        train, test, dicts = pickle.load(f, encoding='bytes') #python3

#Dictionaries and train test partition
w2idx, ne2idx, labels2idx = dicts[b'words2idx'], dicts[b'tables2idx'], dicts[b'labels2idx']
    
idx2w  = dict((v,k) for k,v in w2idx.items())
idx2la = dict((v,k) for k,v in labels2idx.items())

train_x, _, train_label = train
test_x,  _,  test_label  = test


# Max value of word coding to assign the ID_PAD
ID_PAD = np.max([np.max(tx) for tx in train_x]) + 1
print('ID_PAD: ', ID_PAD)

def context(l, size=3):
    l = list(l)
    lpadded = size // 2 * [ID_PAD] + l + size // 2 * [ID_PAD]
    out = [lpadded[i:(i + size)] for i in range(len(l))]
    return out


# Create train and test X y.
X_trn=[]
for s in train_x:
    X_trn += context(s,size=10)
X_trn = np.array(X_trn)

X_tst=[]
for s in test_x:
    X_tst += context(s,size=10)
X_tst = np.array(X_tst)
print('X trn shape: ', X_trn.shape)
print('X_tst shape: ',X_tst.shape)

y_trn=[]
for s in train_label:
    y_trn += list(s)
y_trn = np.array(y_trn)
print('y_trn shape: ',y_trn.shape)

y_tst=[]
for s in test_label:
    y_tst += list(s)
y_tst = np.array(y_tst)
print('y_tst shape: ',y_tst.shape)

print('Num labels: ',len(set(y_trn)))
print('Num words: ',len(set(idx2w)))

ID_PAD:  572
X trn shape:  (56590, 10)
X_tst shape:  (9198, 10)
y_trn shape:  (56590,)
y_tst shape:  (9198,)
Num labels:  121
Num words:  572


# Simpe LSTM model

## Architecture
    - tf.nn.embedding_lookup
    - tf.nn.dynamic_rnn layer
    - Dense layer: tf.nn.relu(tf.matmul(x, W) + b)
    
## Features
    - Dropout
    - Saver
    - Cross entropy with loss regularization
    - Score function

In [3]:
#General parameters
LOG_DIR = '/tmp/tensorboard/airline/embeddings_visualize'

# data attributes
input_seq_length = X_trn.shape[1]
input_vocabulary_size = len(set(idx2w)) + 1
output_length = 127

#Model parameters
embedding_size=64
num_hidden_lstm = 128


In [4]:
# Save words and labels for embedding visualization
if not os.path.exists(LOG_DIR + '/train'):
    os.makedirs(LOG_DIR + '/train')
    
with open( LOG_DIR + '/train/records.tsv', "w") as record_file:
    for item in idx2w.items():
        if sys.version_info.major==2:
            record_file.write(item[1].encode('utf-8')+'\n') #python2.7
        else:
            record_file.write(item[1].decode('ascii')+'\n') #python3



In [5]:
from tensorflow.contrib.tensorboard.plugins import projector

# Define the tensorflow graph

graph = tf.Graph()

with graph.as_default():
    # graph definition
    # Inputs
    with tf.name_scope('Inputs') as scope:
        x = tf.placeholder(tf.int32, shape=[None, input_seq_length], name='x')
        y = tf.placeholder(tf.int64, shape=[None], name='y')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    with tf.name_scope('Embeddings') as scope:
        W_embedding = tf.Variable(tf.random_uniform([input_vocabulary_size, embedding_size], -1.0, 1.0) ,name="W_vis")
        embedding_layer = tf.nn.embedding_lookup(W_embedding, x)
        print('embedding_layer: ', embedding_layer)

    
    with tf.name_scope('RNN') as scope:
        cell_1 = tf.contrib.rnn.LSTMCell(num_hidden_lstm, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=123))
        cell_1 = tf.contrib.rnn.DropoutWrapper(cell_1, output_keep_prob=keep_prob)
        lstm_outputs, lstm_state = tf.nn.dynamic_rnn(cell_1, embedding_layer, dtype=tf.float32)
        print('lstm_outputs: ', lstm_outputs)
 

    #Dense layer form RNN outs to prediction
    with tf.name_scope('Dense') as scope:
        W_dense = tf.Variable(tf.truncated_normal([num_hidden_lstm, output_length], stddev=0.1), name='W_dense')
        b_dense = tf.Variable(tf.constant(0.1, shape=[output_length]), name='b_dense')
        dense_output = tf.nn.relu(tf.matmul(lstm_outputs[:,-1,:], W_dense) + b_dense)
        print('dense_output: ', dense_output)

        

    # Loss function
    with tf.name_scope("xent") as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=dense_output,
                                                                       labels=y, name='cross_entropy')
        ce_summ = tf.summary.histogram("cross entropy", cross_entropy) #TENSORBOARD


    #Optimizer
    with tf.name_scope("train") as scope:
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')
        optimizer = tf.train.AdamOptimizer(learning_rate)
        train_op = optimizer.minimize(cross_entropy, name='train_op')


    #Accuracy
    with tf.name_scope("test") as scope:
        #Prediction
        y_pred = tf.nn.softmax(dense_output, name='y_pred')
        #Accuracy
        correct_prediction = tf.equal(tf.argmax(dense_output,1), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
        accuracy_summary = tf.summary.scalar("accuracy", accuracy) #TENSORBOARD

        
    # Create a saver to save weigths.
    saver = tf.train.Saver()

    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    # Merge all the summaries
    with tf.name_scope('summaries') as scope:
        merged = tf.summary.merge_all()


embedding_layer:  Tensor("Embeddings/embedding_lookup:0", shape=(?, 10, 64), dtype=float32)
lstm_outputs:  Tensor("RNN/rnn/transpose:0", shape=(?, 10, 128), dtype=float32)
dense_output:  Tensor("Dense/Relu:0", shape=(?, 127), dtype=float32)
INFO:tensorflow:Summary name cross entropy is illegal; using cross_entropy instead.


In [6]:
# Create metadata for embedings visualization
w2idx = {v.decode('ascii'): k for k, v in idx2w.items()} 
with open(LOG_DIR + '/train/records.tsv', "w") as record_file:
    record_file.write("UNK\n")
    for item in idx2w.items():
        if sys.version_info.major==3: # python 3
            record_file.write(str(item[1])+'\n')
        else: # Python 2
            record_file.write(item[1].encode('utf-8')+'\n')
            



In [7]:
#batch generator
def batch_generator(x=X_trn, y=y_trn, batch_size=128):
    from sklearn.utils import shuffle
    x_shuffle, y_shuffle = shuffle(x, y, random_state=0)
    for i in range(0, x.shape[0]-batch_size, batch_size):
        x_batch = x_shuffle[i:i+batch_size,:]
        y_batch = y_shuffle[i:i+batch_size]
        yield x_batch, y_batch
    
seq = batch_generator(x=X_trn, y=y_trn, batch_size=20)
print(next(seq))

(array([[554, 241, 481, 165, 193, 197, 208, 379, 502,  64],
       [193, 514, 208,  77, 502, 137, 359, 544,  40, 481],
       [232, 331, 237, 358,  13, 193, 208,  77, 502, 137],
       [ 32, 194,  40, 183, 208, 137, 502, 415, 205, 572],
       [232, 331,  13, 277, 353, 194, 208, 452, 375, 195],
       [572, 193, 348, 208, 313, 502, 282,  71, 358, 249],
       [193, 208, 128, 502, 415, 205, 572, 572, 572, 572],
       [358, 481, 174, 353,  65, 524, 435, 572, 572, 572],
       [208, 481,  29, 234, 379, 502, 159, 572, 572, 572],
       [572, 572, 572, 439, 301, 481, 194, 208, 415, 205],
       [481, 265, 193, 208,  64, 502, 137, 358, 248, 435],
       [157,  37,  26, 221, 561,  13, 105, 353, 430, 111],
       [534, 358, 481, 190, 105,  37,  26, 193, 208, 376],
       [572, 572, 572, 383, 276, 530, 194,  73,  77,  40],
       [572, 572, 572, 572, 554, 194,  50, 389,  86,  37],
       [481, 193, 501, 481, 321, 358, 530,  26, 200, 426],
       [572, 572, 572, 572,  13, 190, 105, 193, 358,  3

In [8]:
# Execute the graph to train a network
import time

batch_size = 256
nEpochs = 20

start = time.time()

gpu_options = tf.GPUOptions(allow_growth = True)
with tf.Session(graph=graph, config=tf.ConfigProto(gpu_options=gpu_options)) as session:

    #Create sumaries writers
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', session.graph, flush_secs=2)
    test_writer  = tf.summary.FileWriter(LOG_DIR + '/test', flush_secs=2)

    
    # Define the embeddings in tensorboard
    config = projector.ProjectorConfig()
    # Create the embedding tensorboard container
    embedding = config.embeddings.add()
    # Link to the embeddings tensor
    embedding.tensor_name = W_embedding.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = LOG_DIR + '/train/records.tsv'
    # Saves a configuration file that TensorBoard will read during startup.
    projector.visualize_embeddings(train_writer, config)

        
    print('Initializing')
    print('Epoch - Loss(trn) -  Acc(trn)   -   Loss(tst) -   Acc(tst)')
    session.run(tf.global_variables_initializer())
    for epoch in range(nEpochs):
        ce_c=[]
        acc_c=[]
        ce_c_tst=[]
        acc_c_tst=[]
        
        batch_list = batch_generator(x=X_trn, y=y_trn, batch_size=batch_size)
        for i, batch in enumerate(batch_list):
            feedDict = {x: batch[0], y: batch[1], keep_prob: 0.5, learning_rate: 0.001} # dictionary of batch data to run the graph
            _, ce, acc = session.run([train_op, cross_entropy, accuracy], feed_dict=feedDict)
            ce_c += [ce]
            acc_c += [acc]
            
        batch_list_tst = batch_generator(x=X_tst, y=y_tst, batch_size=batch_size)
        for x_batch, y_batch in batch_list_tst:
            feedDict = {x: x_batch, y: y_batch, keep_prob: 1} # dictionary of batch data to run the graph
            ce_tst, acc_tst = session.run([cross_entropy, accuracy], feed_dict=feedDict)
            ce_c_tst += [ce_tst]
            acc_c_tst += [acc_tst]
            
        saver.save(session, LOG_DIR + "/train/model.ckpt", epoch)
        
        print(epoch, np.mean(ce_c), np.mean(acc_c), np.mean(ce_c_tst), np.mean(acc_c_tst), sep='   -   ')
print('Time to train:', time.time() - start)
# 235 secs in CPU i7 Mac
# 121 secs in GPU Mac

/home/jorge/anaconda3/envs/py3_tf1/lib/python3.6/site-packages/ipykernel/__main__.py:22: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding
/home/jorge/anaconda3/envs/py3_tf1/lib/python3.6/site-packages/ipykernel/__main__.py:24: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


Initializing
Epoch - Loss(trn) -  Acc(trn)   -   Loss(tst) -   Acc(tst)
0   -   1.89017   -   0.625972   -   1.50744   -   0.643415
1   -   1.19937   -   0.727817   -   1.03019   -   0.770201
2   -   0.801799   -   0.822504   -   0.712798   -   0.853237
3   -   0.546249   -   0.880108   -   0.525917   -   0.892187
4   -   0.386602   -   0.919135   -   0.412727   -   0.914955
5   -   0.296257   -   0.939886   -   0.337433   -   0.934487
6   -   0.236386   -   0.953585   -   0.294111   -   0.942299
7   -   0.196079   -   0.961698   -   0.271015   -   0.94933
8   -   0.171422   -   0.966576   -   0.252969   -   0.955134
9   -   0.153136   -   0.971171   -   0.241621   -   0.956696
10   -   0.137266   -   0.97377   -   0.226541   -   0.959152
11   -   0.120943   -   0.977747   -   0.220152   -   0.959263
12   -   0.111909   -   0.979656   -   0.219351   -   0.960045
13   -   0.102292   -   0.980292   -   0.209216   -   0.959375
14   -   0.090219   -   0.982784   -   0.213229   -   0.96317


In [9]:
# Predict. Score new paragraph 

#inv_map = {v: k for k, v in my_map.iteritems()} #python2

def score_paragraph(paragraph):
    #Preprocess data
    p_w = paragraph.split()
    p_w_c = [w2idx[w] for w in  p_w]
    x_score = np.array(context(p_w_c, size=10))

    with tf.Session(graph=graph) as session:

        saver.restore(session, LOG_DIR + "/train/model.ckpt-19")
        feedDict = {x: x_score, keep_prob: 1} # dictionary of batch data to run the graph
        pred_score = session.run(y_pred, feed_dict=feedDict)

    response = [idx2la[l] for l in np.argmax(pred_score,axis=1)]
    return response


paragraph = 'i need a business ticket in any flight with departure from alaska to las vegas monday with breakfast'
response = score_paragraph(paragraph)
wlength = 35
for wx, la in zip(paragraph.split(), response): print( wx.rjust(wlength), la.rjust(wlength))


                                  i b'                                  O'
                               need b'                                  O'
                                  a b'                                  O'
                           business b'                       B-flight_mod'
                             ticket b'                                  O'
                                 in b'                                  O'
                                any b'                                  O'
                             flight b'                                  O'
                               with b'                                  O'
                          departure b'                                  O'
                               from b'                                  O'
                             alaska b'                                  O'
                                 to b'                B-fromloc.city_name'
                         